## 1. Data Engineering - Process CSV files into BQ Tables via Spark

### Create Spark session with BQ connector

Create a Spark session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType

from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName('Spark - Data Eng Demo') \
.config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest.jar') \
.getOrCreate()

**TODO**
* Provide your bucket path below


In [ ]:
!gsutil ls <bucket-path>

Check the first 1000 bytes of a file on GCS

In [ ]:
!gsutil cat -h -r 0-1000 <bucket-path>/transaction_data_train.csv

In [ ]:
path_to_train_csv = "<bucket-path>/transaction_data_train.csv"

### Get Spark application ID 

This is useful to easily fine application in the Spark History UI

In [ ]:
spark.conf.get("spark.app.id")

Load the CSV file into a Spark Dataframe

In [ ]:
df_transaction_data_from_csv = spark \
.read \
.option("inferSchema" , "true") \
.option("header" , "true") \
.csv(path_to_train_csv)

In [ ]:
df_transaction_data_from_csv.printSchema()

### Create the BQ dataset & table to persist data

**TODO** (Challenge 1)
* Create the BQ schema from the spark dataframe 
    * Reference for converting data types: https://github.com/GoogleCloudDataproc/spark-bigquery-connector#data-types

In [ ]:
schema_inline = <insert-code-here>
schema_inline

In [ ]:
df_transaction_data_from_csv.show(5)

In [ ]:
# create the name your BQ dataset
project_id = !gcloud config list --format 'value(core.project)' 2>/dev/null 
dataset_name = project_id[0] + '-raw'
dataset_name = dataset_name.replace('-', '_')
dataset_name

Create the BQ dataset by specifying the location

In [ ]:
!bq --location=europe-west3 mk -d \
{dataset_name}

In [ ]:
# create path to new table for creation
bq_table_name = 'transaction_data_train'
bq_table_path= dataset_name + '.' + bq_table_name
bq_table_path

Create the BQ table 

In [ ]:
!bq mk --table \
{bq_table_path} \
{schema_inline}

#### Check that table was created

In [ ]:
table = project_id[0] + ":" + bq_table_path
df_transaction_data_from_bq = spark.read \
.format("bigquery") \
.option("table", table) \
.load()

In [ ]:
df_transaction_data_from_bq.printSchema()

In [ ]:
df_transaction_data_from_bq.show()

Write spark dataframe to BQ table

In [ ]:
# create temp GCS bucket for writing spark df to bq table
gcs_bucket = project_id[0] + '-data'
gcs_bucket

In [ ]:
df_transaction_data_from_csv.write \
.format("bigquery") \
.option("table", table) \
.option("temporaryGcsBucket", gcs_bucket) \
.mode('overwrite') \
.save()

Check if the BQ table is populated 

In [ ]:
df_transaction_data_from_bq = spark.read \
.format("bigquery") \
.option("table", table) \
.load()

In [ ]:
df_transaction_data_from_bq.show()

### Compute statistics for columns in table

In [ ]:
df_transaction_data_from_bq.describe().show()